In [77]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model

In [78]:
location_model = load_model('CNN_location_model.keras')

In [79]:
def normalize_test_image():
    img_normalized = []
    folder_path = "./test_image"

    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)
        img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        img_normalized.append(img)

    return np.array(img_normalized)

In [80]:
test_image_data = normalize_test_image()
test_image_data.shape

(1, 208, 267)

In [81]:
cropped_image_size = 80

test_data = []
test_data_coordinate = []
for img in test_image_data:
    for i in range(0, 186, 2):
        for j in range(0, 127, 2):
            test_image = img[i:i+cropped_image_size, j:j+cropped_image_size]
            test_image = cv2.resize(test_image, dsize=(50, 50))
            test_data.append(test_image)
            test_data_coordinate.append([j+int(cropped_image_size / 2), i+int(cropped_image_size / 2)])

In [82]:
test_data = np.array(test_data)

In [83]:
test_pred = location_model.predict(test_data)
test_pred

186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


array([[4.6279720e-24],
       [7.5609020e-16],
       [6.2626341e-25],
       ...,
       [6.7728586e-20],
       [8.4373155e-14],
       [6.1951153e-11]], dtype=float32)

In [84]:
test_pred = test_pred.flatten()

In [85]:
def group_assign_weights(arr, threshold=3):
    
    arr = np.sort(arr)
    weights = {}
    current_group = [arr[0]]

    for i in range(1, len(arr)):
        if arr[i] - arr[i-1] <= threshold:
            current_group.append(arr[i])
        else:
            weights[current_group[0]] = len(current_group)
            current_group = [arr[i]]

    weights[current_group[0]] = len(current_group)

    return weights

In [86]:
index_weight_dictionary = group_assign_weights(np.argsort(test_pred)[-10:])
final_index = max(index_weight_dictionary, key=index_weight_dictionary.get)

In [87]:
def show_predict_result(test_data_coordinate, center_x, center_y):

    # 이미지 불러오기 (흑백 이미지)
    image = test_image_data[0]

    # 흑백 이미지를 3채널 이미지로 변환 (BGR)
    image_colored = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # 사각형 크기 설정
    rect_size = 50
    half_size = rect_size // 2

    # 사각형의 왼쪽 상단 좌표 (top-left)와 오른쪽 하단 좌표 (bottom-right) 계산
    top_left = (center_x - half_size, center_y - half_size)
    bottom_right = (center_x + half_size, center_y + half_size)

    # 빨간색 사각형 그리기 (색상은 BGR로 설정, 빨간색은 (0, 0, 255))
    cv2.rectangle(image_colored, top_left, bottom_right, (0, 0, 255), 2)

    # 결과 이미지 보기
    cv2.imshow("predicted", image_colored)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [88]:
center_x = test_data_coordinate[final_index][0]
center_y = test_data_coordinate[final_index][1]

show_predict_result(test_data_coordinate, center_x, center_y)

In [89]:
rotation_model = load_model('CNN_rotation_model.keras')

In [98]:
crop_size = 80

half_crop_size = crop_size // 2
            
start_x = center_x - half_crop_size
start_y = center_y - half_crop_size
end_x = center_x + half_crop_size
end_y = center_y + half_crop_size

final_block_image = test_image_data[0][start_y:end_y, start_x:end_x]
final_block_image = np.array([cv2.resize(final_block_image, dsize=(50, 50))])

In [99]:
final_block_image.shape

(1, 50, 50)

In [106]:
cv2.imshow('img', final_block_image[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [101]:
rotation_model.predict(final_block_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[9.9999726e-01, 2.1305618e-06, 5.4541056e-07]], dtype=float32)

In [102]:
rotation_pred = np.argmax(rotation_model.predict(final_block_image)) * 30
rotation_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


0

In [107]:
print('final model prediction')
print('block location')
print(f'x : {center_x}, y : {center_y}')
print(f'block rotated : {rotation_pred} degree')

final model prediction
block location
x : 150, y : 40
block rotated : 0 degree
